In [2]:
x_train, y_train, x_test, y_test = utils.get_training_data(0.2, onehot=0, standardize=True)

In [42]:
# this is the size of our encoded representations
encoding_dim = 100

# this is our input placeholder
input_features = Input(shape=(x_train.shape[1],))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_features)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(x_train.shape[1], activation='relu')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_features, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

encoder = Model(input_features, encoded)

# create a placeholder for an encoded (100-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 2s 102us/step - loss: 3.3372 - val_loss: 2.8671
Epoch 2/50
16000/16000 [==============================] - 2s 95us/step - loss: 2.5287 - val_loss: 2.1990
Epoch 3/50
16000/16000 [==============================] - 2s 94us/step - loss: 2.0002 - val_loss: 1.8503
Epoch 4/50
16000/16000 [==============================] - 1s 93us/step - loss: 1.7527 - val_loss: 1.6951
Epoch 5/50
16000/16000 [==============================] - 2s 95us/step - loss: 1.6369 - val_loss: 1.6173
Epoch 6/50
16000/16000 [==============================] - 1s 91us/step - loss: 1.5762 - val_loss: 1.5753
Epoch 7/50
16000/16000 [==============================] - 2s 94us/step - loss: 1.5403 - val_loss: 1.5502
Epoch 8/50
16000/16000 [==============================] - 1s 91us/step - loss: 1.5179 - val_loss: 1.5332
Epoch 9/50
16000/16000 [==============================] - 1s 91us/step - loss: 1.5020 - val_loss: 1.5233
Epoch

In [43]:
# Create a new training set based on the encoder predictions

x_train_encoded = encoder.predict(x_train)
x_test_encoded = encoder.predict(x_test)

In [47]:
num_estimators = 1000
classifier = RandomForestClassifier(n_estimators=num_estimators, max_depth=10, min_samples_leaf=5)
classifier.fit(x_train_encoded, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
classifier.score(x_train_encoded, y_train)

0.9594375

In [49]:
classifier.score(x_test_encoded, y_test)

0.7375